Optuna kann Hyperparameter basierend auf Algorithmen selbstständig anpassen

In [1]:
import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState

from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop

from tf_utils.mnistData_advance import MNIST

In [2]:
BATCHSIZE = 128
CLASSES = 10
EPOCHS = 20

In [3]:
def create_model(trial):
    n_layers = trial.suggest_int("n_layers", 1, 3)
    model = Sequential()
    model.add(Flatten())
    for i in range(n_layers):
        num_hidden = trial.suggest_int(f"n_units_l{i}", 4, 128, log=True)
        model.add(Dense(num_hidden, activation="relu"))
        dropout = trial.suggest_float(f"dropout_l{i}", 0.2, 0.5)
        model.add(Dropout(rate=dropout))
    model.add(Dense(CLASSES, activation="softmax"))

    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)  # 0.00001 0.1
    model.compile(
        loss="categorical_crossentropy",
        optimizer=RMSprop(learning_rate=learning_rate),
        metrics=["accuracy"],
    )

    return model

In [4]:
def objective(trial):
    data = MNIST()

    train_dataset = data.get_train_set()
    val_dataset = data.get_val_set()
    test_dataset = data.get_test_set()

    model = create_model(trial)

    model.fit(
        x=train_dataset,
        batch_size=BATCHSIZE,
        callbacks=[TFKerasPruningCallback(trial, "val_accuracy")],
        epochs=EPOCHS,
        validation_data=val_dataset,
        verbose=1,
    )

    score = model.evaluate(x=test_dataset, verbose=0)
    return score[1]

In [5]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=100)
    pruned_trials = study.get_trials(deepcopy=False, states=(TrialState.PRUNED,))
    complete_trials = study.get_trials(deepcopy=False, states=(TrialState.COMPLETE,))
    print("Study statistics: ")
    print(f"  Number of finished trials: {len(study.trials)}")
    print(f"  Number of pruned trials: {len(pruned_trials)}")
    print(f"  Number of complete trials: {len(complete_trials)}")

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

[I 2022-03-28 17:35:02,535] A new study created in memory with name: no-name-cea9afc4-bd4e-4283-a664-9f33c1082b3f


Epoch 1/20
315/315 [==============================] - 4s 5ms/step - loss: 2.3259 - accuracy: 0.1148 - val_loss: 2.2513 - val_accuracy: 0.1831
Epoch 2/20
315/315 [==============================] - 2s 5ms/step - loss: 2.2662 - accuracy: 0.1445 - val_loss: 2.2018 - val_accuracy: 0.2624
Epoch 3/20
315/315 [==============================] - 2s 5ms/step - loss: 2.2171 - accuracy: 0.1768 - val_loss: 2.1478 - val_accuracy: 0.3155
Epoch 4/20
315/315 [==============================] - 2s 5ms/step - loss: 2.1723 - accuracy: 0.2017 - val_loss: 2.0886 - val_accuracy: 0.3554
Epoch 5/20
315/315 [==============================] - 2s 5ms/step - loss: 2.1244 - accuracy: 0.2299 - val_loss: 2.0224 - val_accuracy: 0.3896
Epoch 6/20
315/315 [==============================] - 2s 5ms/step - loss: 2.0829 - accuracy: 0.2486 - val_loss: 1.9595 - val_accuracy: 0.4216
Epoch 7/20
315/315 [==============================] - 2s 5ms/step - loss: 2.0413 - accuracy: 0.2699 - val_loss: 1.9021 - val_accuracy: 0.4515
Epoch 

[I 2022-03-28 17:35:40,038] Trial 0 finished with value: 0.699400007724762 and parameters: {'n_layers': 2, 'n_units_l0': 21, 'dropout_l0': 0.4283869271209061, 'n_units_l1': 17, 'dropout_l1': 0.3898638826977123, 'learning_rate': 1.497182787574073e-05}. Best is trial 0 with value: 0.699400007724762.


Epoch 1/20
315/315 [==============================] - 2s 6ms/step - loss: 1.6119 - accuracy: 0.3914 - val_loss: 1.0609 - val_accuracy: 0.6289
Epoch 2/20
315/315 [==============================] - 2s 5ms/step - loss: 1.3869 - accuracy: 0.4732 - val_loss: 1.0358 - val_accuracy: 0.5688
Epoch 3/20
315/315 [==============================] - 2s 5ms/step - loss: 1.3289 - accuracy: 0.5158 - val_loss: 1.0398 - val_accuracy: 0.6068
Epoch 4/20
315/315 [==============================] - 1s 4ms/step - loss: 1.3137 - accuracy: 0.5292 - val_loss: 1.0665 - val_accuracy: 0.5845
Epoch 5/20
315/315 [==============================] - 1s 5ms/step - loss: 1.3123 - accuracy: 0.5286 - val_loss: 1.2186 - val_accuracy: 0.5416
Epoch 6/20
315/315 [==============================] - 1s 5ms/step - loss: 1.3054 - accuracy: 0.5361 - val_loss: 1.0878 - val_accuracy: 0.5875
Epoch 7/20
315/315 [==============================] - 1s 5ms/step - loss: 1.3109 - accuracy: 0.5333 - val_loss: 1.0679 - val_accuracy: 0.5970
Epoch 

[I 2022-03-28 17:36:12,160] Trial 1 finished with value: 0.5576000213623047 and parameters: {'n_layers': 3, 'n_units_l0': 12, 'dropout_l0': 0.46536841263115114, 'n_units_l1': 57, 'dropout_l1': 0.34063404744204595, 'n_units_l2': 65, 'dropout_l2': 0.29246939684923234, 'learning_rate': 0.007799498658855895}. Best is trial 0 with value: 0.699400007724762.


...

Epoch 1/20
310/315 [============================>.] - ETA: 0s - loss: 0.4914 - accuracy: 0.8590

[I 2022-03-28 17:56:35,321] Trial 98 pruned. Trial was pruned at epoch 0.


Epoch 1/20
315/315 [==============================] - 2s 4ms/step - loss: 0.7839 - accuracy: 0.7454 - val_loss: 0.2374 - val_accuracy: 0.9339
Epoch 2/20
303/315 [===========================>..] - ETA: 0s - loss: 0.4404 - accuracy: 0.8699

[I 2022-03-28 17:56:39,306] Trial 99 pruned. Trial was pruned at epoch 1.


Study statistics: 
  Number of finished trials: 100
  Number of pruned trials: 58
  Number of complete trials: 42
Best trial:
  Value:  0.9789000153541565
  Params: 
    n_layers: 1
    n_units_l0: 125
    dropout_l0: 0.28155932664292166
    learning_rate: 0.003380608431168201
